In [1]:
import os
import datetime
import json
import numpy as np
import pandas as pd
import pprint

from IPython.display import display, HTML
from pymongo import MongoClient

In [2]:
# Connect to Mongo host & port

client = MongoClient(os.environ['MONGODB_NAME'], 27017)

In [3]:
# Input the name of the database you'd like to connect to. Example 'sacred_demo' or 'pythia_experiment'

db_name = input()

sacred_demo


In [4]:
# Connect to db

db = client.get_database(db_name)

In [5]:
# This is the date of the experiments you'd like to see. Right now it is set at any experiment run after this date

date = datetime.datetime(2016, 7, 28, 0, 0, 0, 0)

In [6]:
# This mongo query displays the results and the arguments passed in to the model

cursor = db.default.runs.find({'start_time': {"$gt" : date}}, {'config': True, 'result': True, '_id': False})
count = 0
for doc in cursor:
        count += 1
        print("\n-----------------------------")
        print("RUN NUMBER: " + str(count))
        print("-----------------------------")
        for key in doc['config'].keys():
            if key == 'args':
                result = {}
                # Print data used for the experiment
                args = doc['config']['args']
                print("\nData: " + str(args[0]) + "\n")
                # The way the config is set up, the first index is the data used and the remaining ones are the args
                for d in args[1:len(args)]:
                    result.update(d)
                df = pd.DataFrame(result).fillna(" ")
                key_list = []
                # I combine the dictionaries to make a nicer display for the pandas dataframe
                for key in result.keys():
                    for key2 in (result[key].keys()):
                        key_list.append(key2)
                args_df = df.reindex(key_list)
                display(args_df)
            else:
                print("Running with a seed of: " + str(doc['config']['seed']) + "\n")
        print("Results are:")
        display(pd.DataFrame(doc['result']))


-----------------------------
RUN NUMBER: 1
-----------------------------
Running with a seed of: 403041931

Results are:


,accuracy,confusion matrix,f score,mae,precision,recall,rmse,support
0,0.638655,"[14, 17]",0.394366,0.361345,0.35000,0.451613,0.601119,31
1,0.638655,"[26, 62]",0.742515,0.361345,0.78481,0.704545,0.601119,88



-----------------------------
RUN NUMBER: 2
-----------------------------
Running with a seed of: 78345414


Data: /Users/davec/pythia-12July/src/data/stackexchange/stack_exchange_data/corpus_filtered/anime



,bow,log_reg,resampling,svm,xgb
log_penalty,,l2,,,
log_tol,,0.0001,,,
log_C,,0.0001,,,
cos,True,,,,
product,True,,,,
tfidf,True,,,,
difference,True,,,,
svm_kernal,,,,linear,
svm_gamma,,,,auto,1
svm_C,,,,2000,


Results are:


,accuracy,confusion matrix,f score,mae,precision,recall,rmse,support
0,0.680672,"[17, 14]",0.472222,0.319328,0.414634,0.548387,0.565091,31
1,0.680672,"[24, 64]",0.771084,0.319328,0.820513,0.727273,0.565091,88


In [7]:
# The same concept as earlier: display experiments after the specified date below

date_pythia = datetime.datetime(2016, 8, 16, 0, 0, 0, 0)

In [8]:
# Enter the name of the db you would like to see

db_name = input()

pythia_experiment


In [9]:
# Connect to db

db = client.get_database(db_name)

In [13]:
# This is for the logic behind the visualization. If all elements within the lists are false, 
# then the algorithm was not used

LDA_list = ['LDA_APPEND', 'LDA_COS', 'LDA_DIFFERENCE', 'LDA_PRODUCT', 'LDA_TOPICS']
BOW_list = ['BOW_APPEND', 'BOW_PRODUCT', 'BOW_COS', 'BOW_DIFFERENCE', 'BOW_TFIDF']
W2V_list = ['W2V_APPEND', 'W2V_DIFFERENCE', 'W2V_COS', 'W2V_PRODUCT']
W2V_associated_list = ['W2V_SIZE', 'W2V_WINDOW', 'W2V_WORKERS', 'W2V_MIN_COUNT', 'W2V_PRETRAINED']
CNN_list = ['CNN_APPEND', 'CNN_COS', 'CNN_PRODUCT', 'CNN_DIFFERENCE']
ST_list = ['ST_APPEND', 'ST_COS', 'ST_PRODUCT', 'ST_DIFFERENCE']
SVM_list = ['SVM']
WORDONEHOT_list = ['WORDONEHOT']
XGB_list = ['XGB']
LOG_REG_list = ['LOG_REG']

In [14]:
cursor = db.default.runs.find({'start_time': {"$gt" : date_pythia}}, {'config': True, 'result': True, '_id': False})
count = 0

bool_listLDA = []
bool_listCNN = []
bool_listST = []
bool_listW2V = []
bool_listBOW = []

bool_listSVM = []
bool_listXGB = []
bool_listLOG_REG = []
bool_listWORDONEHOT = []

for doc in cursor:
    count += 1
    print("-------------------------")
    print("RUN NUMBER: " + str(count))
    print("-------------------------")
    df = pd.DataFrame(doc['config'], index=['value']).T
    
    for item in CNN_list:
        bool_listCNN.append(df.get_value(item, 'value'))
    if not any(bool_listCNN) == True:
            df = df.drop(CNN_list)
            if 'CNN_CHAR_VOCAB' in df.index:
                df = df.drop(['CNN_CHAR_VOCAB'])
            if 'CNN_VOCAB_TYPE' in df.index:
                df = df.drop(['CNN_VOCAB_TYPE'])
    
    for item in LDA_list:
        bool_listLDA.append(df.get_value(item, 'value'))
    if not any(bool_listLDA) == True:
        df = df.drop(LDA_list)
    
    for item in ST_list:
        bool_listST.append(df.get_value(item, 'value'))
    if not any(bool_listST) == True:
        df = df.drop(ST_list)
    
    for item in W2V_list:
        bool_listW2V.append(df.get_value(item, 'value'))
    if not any(bool_listW2V) == True:
        df = df.drop(W2V_list)
        for item in W2V_associated_list:
            if item in df.index:
                df = df.drop([item])
                
    for item in BOW_list:
        bool_listBOW.append(df.get_value(item, 'value'))
    if not any(bool_listBOW) == True:
        df = df.drop(BOW_list)        
        
    for item in SVM_list:
        bool_listSVM.append(df.get_value(item, 'value'))
    if not any(bool_listSVM) == True:
        df = df.drop(['SVM', 'SVM_C', 'SVM_GAMMA', 'SVM_KERNEL'])
        
    for item in XGB_list:
        bool_listXGB.append(df.get_value(item, 'value'))
    if not any(bool_listXGB) == True:
        df = df.drop(['XGB', 'XGB_COLSAMPLEBYTREE', 'XGB_LEARNRATE', 'XGB_MAXDEPTH', 'XGB_MINCHILDWEIGHT'])
    
    for item in LOG_REG_list:
        bool_listLOG_REG.append(df.get_value(item, 'value'))
    if not any(bool_listLOG_REG) == True:
        df = df.drop(['LOG_REG', 'LOG_PENALTY', 'LOG_C', 'LOG_TOL'])
        
    for item in WORDONEHOT_list:
        bool_listWORDONEHOT.append(df.get_value(item, 'value'))
    if not any(bool_listWORDONEHOT) == True:
        df = df.drop(['WORDONEHOT', 'WORDONEHOT_VOCAB'])
        
    display(df)
    print("\nRESULTS ARE:\n")
    display(pd.DataFrame(doc['result']))

-------------------------
RUN NUMBER: 1
-------------------------


,value
BOW_APPEND,True
BOW_COS,False
BOW_DIFFERENCE,False
BOW_PRODUCT,True
BOW_TFIDF,False
LDA_APPEND,True
LDA_COS,False
LDA_DIFFERENCE,True
LDA_PRODUCT,False
LDA_TOPICS,50



RESULTS ARE:



,accuracy,confusion matrix,f score,mae,precision,recall,rmse,support
0,0.619883,"[8, 24]",0.197531,0.380117,0.163265,0.250000,0.616536,32
1,0.619883,"[41, 98]",0.750958,0.380117,0.803279,0.705036,0.616536,139


-------------------------
RUN NUMBER: 2
-------------------------


...

,accuracy,confusion matrix,f score,mae,precision,recall,rmse,support
0,0.709677,"[9, 36]",0.222222,0.290323,0.250000,0.200000,0.538816,45
1,0.709677,"[27, 145]",0.821530,0.290323,0.801105,0.843023,0.538816,172


-------------------------
RUN NUMBER: 45
-------------------------


,value
BOW_APPEND,True
BOW_COS,False
BOW_DIFFERENCE,False
BOW_PRODUCT,False
BOW_TFIDF,True
CNN_APPEND,False
CNN_CHAR_VOCAB,abcdefghijklmnopqrstuvwxyz0123456789
CNN_COS,False
CNN_DIFFERENCE,False
CNN_PRODUCT,False



RESULTS ARE:



KeyError: 'result'

In [12]:
cursor = db.default.runs.find({ "experiment.name": "pythia_gridsearch"})

for doc in cursor:
    print(doc)

{'config': {'seed': 109702043, 'logreg_params': {'tol': [0.01, 0.001, 0.0001, 0.0001, 1e-05], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l1', 'l2']}, 'logreg_search': False, 'svm_params': {'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['auto', 0.01, 0.001, 0.0001, 0.0001], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, 'xgb_params': {'max_depth': [3, 5, 10, 50, 100], 'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.5], 'min_child_weight': [1, 2, 5, 10, 50]}, 'targetfile': 'target.pkl', 'datafile': 'data.pkl', 'svm_search': False, 'xgb_search': False}, 'resources': [], 'captured_out': '', 'info': {}, 'start_time': datetime.datetime(2016, 8, 16, 17, 27, 8, 382000), 'stop_time': datetime.datetime(2016, 8, 16, 17, 27, 8, 427000), 'host': {'python_version': '3.5.2', 'os_info': 'Darwin-14.5.0-x86_64-i386-64bit', 'python_compiler': 'GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)', 'cpu_count': 4, 'hostname': 'TookieClothespin.local', 'os': 'Darwin', 'cpu': 'Intel(R) Core(TM) i7-5557